In [1]:
"pcsk_76oyA9_6WW7Bbmxb12QwSaDA1M7ycmZg9tBm1Vh3htqneoJuh8joc2XoAgJBdvMdVjDNbd"

'pcsk_76oyA9_6WW7Bbmxb12QwSaDA1M7ycmZg9tBm1Vh3htqneoJuh8joc2XoAgJBdvMdVjDNbd'

In [2]:
PINECONE_API_KEY = "pcsk_76oyA9_6WW7Bbmxb12QwSaDA1M7ycmZg9tBm1Vh3htqneoJuh8joc2XoAgJBdvMdVjDNbd"
INDEX_NAME = "quickstart"

In [3]:
!pip install -q pinecone sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.9/745.9 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.9/280.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.6 MB/s eta 0:00:00


In [4]:
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer
import time


In [5]:

pc = Pinecone(api_key=PINECONE_API_KEY)


In [6]:
if INDEX_NAME not in pc.list_indexes().names():
    pc.create_index(
        name=INDEX_NAME,
        dimension=384,  # embedding size
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

# wait for index to be ready
time.sleep(5)


In [7]:
model = SentenceTransformer("all-MiniLM-L6-v2")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
index = pc.Index(INDEX_NAME)


In [9]:
documents = [
    "Artificial intelligence is transforming healthcare",
    "Machine learning is a subset of AI",
    "Pinecone is a vector database",
    "Deep learning uses neural networks",
    "Natural language processing deals with text data"
]


In [10]:
embeddings = model.encode(documents).tolist()


In [11]:
vectors = []

for i, emb in enumerate(embeddings):
    vectors.append({
        "id": str(i),
        "values": emb,
        "metadata": {"text": documents[i]}
    })

index.upsert(vectors=vectors)


UpsertResponse(upserted_count=5, _response_info={'raw_headers': {'date': 'Mon, 09 Feb 2026 06:36:57 GMT', 'content-type': 'application/json', 'content-length': '19', 'connection': 'keep-alive', 'x-pinecone-request-lsn': '1', 'x-pinecone-request-logical-size': '7940', 'x-pinecone-request-latency-ms': '374', 'x-envoy-upstream-service-time': '253', 'x-pinecone-response-duration-ms': '376', 'grpc-status': '0', 'server': 'envoy'}})

In [12]:
query = "What is a vector database?"

query_embedding = model.encode(query).tolist()

results = index.query(
    vector=query_embedding,
    top_k=3,
    include_metadata=True
)


In [13]:
for match in results["matches"]:
    print("Score:", round(match["score"], 3))
    print("Text:", match["metadata"]["text"])
    print("-" * 40)


Score: 0.615
Text: Pinecone is a vector database
----------------------------------------
Score: 0.26
Text: Natural language processing deals with text data
----------------------------------------
Score: 0.258
Text: Deep learning uses neural networks
----------------------------------------
